In [1]:
import deepchem as dc
import pandas as pd
import numpy as np

import xgboost as xgb
from sklearn.model_selection import GridSearchCV,KFold

from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import matthews_corrcoef

Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (D:\anaconda\envs\PI3K\lib\site-packages\deepchem\models\torch_models\__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [2]:
data = pd.read_csv('../../data/refined_gabaa.csv')

# Feature extraction & Data splitting

In [3]:
featurizer = dc.feat.MACCSKeysFingerprint()
X = featurizer.featurize(data['smiles'])
Y = data['label']
dataset = dc.data.DiskDataset.from_numpy(X=X,y=Y,ids=data['smiles'])

In [6]:
splitter = dc.splits.ScaffoldSplitter()
train_dataset,test_dataset = splitter.train_test_split(dataset=dataset,frac_train=0.7,seed=200)

# GridSearchCV

In [7]:
scoring = {
          'F1':make_scorer(f1_score),
          'AUC':make_scorer(roc_auc_score),
           'BA':make_scorer(balanced_accuracy_score),
           'MCC':make_scorer(matthews_corrcoef)
}


param_grid = {'n_estimators':[200],
                  'max_depth':[3,5,7,9],
                  'colsample_bytree':[0.5,0.7,0.9],
                 'learning_rate':[0.01,0.05, 0.1,0.15,0.2]
                 }

xgb_classifier = xgb.XGBClassifier()

cv = KFold(n_splits=5, shuffle=True, random_state=200)

gs = GridSearchCV(
                   xgb_classifier,
                   param_grid,
                 scoring = scoring,
                  cv = cv,
                 n_jobs = -1,
               refit = 'F1',
               return_train_score = True)

gs_fit = gs.fit(train_dataset.X, train_dataset.y.ravel())

D:\anaconda\envs\PI3K\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:26:01] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [8]:
val_F1 = gs.best_score_
val_AUC = gs.cv_results_['mean_test_AUC'][gs.best_index_]
val_BA = gs.cv_results_['mean_test_BA'][gs.best_index_]
val_MCC = gs.cv_results_['mean_test_MCC'][gs.best_index_]

# GridSearchCV_Result

In [9]:
print('Best parameters: ', gs.best_params_)
print('Best score (F1): ', gs.best_score_)
print('AUC: ',gs.cv_results_['mean_test_AUC'][gs.best_index_])
print('BA: ', gs.cv_results_['mean_test_BA'][gs.best_index_])
print('MCC: ',gs.cv_results_['mean_test_MCC'][gs.best_index_])

Best parameters:  {'colsample_bytree': 0.5, 'learning_rate': 0.15, 'max_depth': 9, 'n_estimators': 200}
Best score (F1):  0.8127029010346719
AUC:  0.8593102963326789
BA:  0.8593102963326789
MCC:  0.7256191218627507


In [10]:
xgb_model = gs_fit.best_estimator_
xgb_model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5,
              enable_categorical=False, gamma=0, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.15, max_delta_step=0, max_depth=9,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=200, n_jobs=8, num_parallel_tree=1, predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

# Evaluate model

In [11]:
y_test_pred_proba = xgb_model.predict_proba(test_dataset.X)
y_test_pred = xgb_model.predict(test_dataset.X)

In [12]:
tn, fp, fn, tp = confusion_matrix(test_dataset.y, y_test_pred).ravel()
print('TN:', tn)
print('FP:', fp)
print('FN:', fn)
print('TP:', tp)

TN: 158
FP: 20
FN: 130
TP: 265


In [13]:
test_pred_list = []
for test_score in y_test_pred_proba:
    test_score = test_score[1]
    test_pred_list.append(test_score)

In [14]:
test_pred_array = np.array(test_pred_list)

In [15]:
test_F1 = f1_score(test_dataset.y,np.round(test_pred_array))
test_AUC = roc_auc_score(test_dataset.y,test_pred_list)
test_BA =  balanced_accuracy_score(test_dataset.y,np.round(test_pred_array))
test_MCC = matthews_corrcoef(test_dataset.y,np.round(test_pred_array))

# Finall result

In [16]:
performance_dataset = {
    'F1':[val_F1,test_F1],
    'AUC':[val_AUC,test_AUC],
    'BA':[val_BA,test_BA],
    'MCC':[val_MCC,test_MCC],
}

In [17]:
performance = pd.DataFrame(performance_dataset,index=['val','test'])
performance

,F1,AUC,BA,MCC
val,0.812703,0.859310,0.859310,0.725619
test,0.779412,0.884085,0.779263,0.516932
